In [ ]:
import pandas as pd

import os
import pickle
from time import time

import pandas as pd
import torch
import torch.nn as nn
from peft import LoraConfig, TaskType, get_peft_config, get_peft_model
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, RandomSampler
from transformers import (AdamW, LlamaForSequenceClassification,
                          LlamaTokenizer, Trainer, TrainingArguments,
                          get_linear_schedule_with_warmup)

from llama_dataloader import BinaryClassificationDataset

In [2]:
data = "data/train_data_01_4_200.csv"

In [3]:
train_df = pd.read_csv(data, delimiter="$")

In [4]:
train_df

,search_id,query,resume,label
0,1b42c480-7599-11ed-b4a7-237a04cf3ba2,quality analyst or quality assurance analyst o...,title: qa specialist^years: 2-4^location: omah...,1
1,1b42c480-7599-11ed-b4a7-237a04cf3ba2,quality analyst or quality assurance analyst o...,"title: test analyst, sdk, game, service^years:...",0
2,3f10fc80-662e-11ed-b10e-2bd97e927a21,internal medicine or internal medicine physici...,title: hospitalist^years: 6-8^location: greenv...,0
3,3f10fc80-662e-11ed-b10e-2bd97e927a21,internal medicine or internal medicine physici...,title: hospitalist^years: 2-4^location: greenv...,1
4,6fc12df0-7a10-11ed-adf0-051792cd6d2d,product designer or design lead or lead design...,title: product designer^years: 8-10^location: ...,1
...,...,...,...,...
44455,sh_fff88430-ad4c-11ed-8dc8-2ff97fcb4fb4,rn or registered nurse or staff rn or staff nu...,title: registered nurse^years: 0-2^location: w...,1
44456,sh_fffa0ae0-b3db-11ed-bf57-cdc2479c45f6,"tax or entrepreneur or security, with 2-4 or 4...","title: research associate, financial, security...",0
44457,sh_fffa0ae0-b3db-11ed-bf57-cdc2479c45f6,"tax or entrepreneur or security, with 2-4 or 4...","title: finance manager, financial, security, p...",1
44458,sh_fffe7550-b3a7-11ed-b253-c7d70a6eb095,"compliance, with 4-6 or 6-8 or 8-10 years of e...","title: business, attorney^years: 8-10^location...",0


In [5]:
train_df['jd_reusme'] = train_df['query'] + "\n" + train_df['resume']
train_df = train_df.dropna(axis=0)

In [6]:
nan_age_df = train_df[train_df['jd_reusme'].isna()]
nan_age_df

,search_id,query,resume,label,jd_reusme


In [7]:
train_data, valid_data = train_test_split(train_df, test_size=0.2, random_state=42)
train_texts, train_labels = train_data['jd_reusme'].tolist(), train_data['label'].tolist()
valid_texts, valid_labels = valid_data['jd_reusme'].tolist(), valid_data['label'].tolist()

In [8]:
tokenizer = LlamaTokenizer.from_pretrained("/home/ubuntu/ranking_llama/weights/tokenizer/")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

1

In [9]:
MAX_LENGTH = 512
train_dataset = BinaryClassificationDataset(train_data, tokenizer, max_length=MAX_LENGTH)
valid_dataset = BinaryClassificationDataset(valid_data, tokenizer, max_length=MAX_LENGTH)

In [10]:
batch_size=1
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

In [11]:
for i, batch in enumerate(train_dataloader):
    print(f"Batch {i}: {batch}")
    break

Batch 0: {'input_ids': tensor([[    1,  7047, 22055,   470, 25001,  3136,   286, 29980, 17191,   470,
          1442, 22055,   470,  7047, 13897,   470,  1442, 13897, 29892,   411,
         29871, 29906, 29899, 29946,   470, 29871, 29946, 29899, 29953,   470,
         29871, 29953, 29899, 29947,   470, 29871, 29947, 29899, 29896, 29900,
           470, 29871, 29896, 29900, 29899, 29896, 29945,   470, 29871, 29896,
         29945, 29899, 29906, 29900,  2440,   310,  7271, 29889,  1818,   367,
          9985,   411,   413, 13961, 29889,   297,   278, 10161,   310,   592,
         13821,  4272,   470,   592, 13821,  4272,  4038, 29892,   592, 13821,
           470,   592, 13821,  4272, 29892,   592, 13821,  4272, 29892,   592,
         13821, 29889,    13,  3257, 29901,  7047, 22055, 29892,  1442, 29985,
          6360, 29879, 29901, 29871, 29906, 29899, 29946, 29985,  5479, 29901,
           592, 13821,  4272, 29985, 14518,  2159, 29901, 29871, 29896, 29900,
         29900, 29896, 29899,